- 시작 하기 전에

이 코드는 python 3.10 버전 기준으로 돌아가는 코드입니다. 해당 버전에서 아래 라이브러리들을 전부 설치해주셔야 돌아갑니다

그리고 처음부터 전부 작동시키면 하루정도 걸려야 다 돌아가니 가급적 제가드린 데이터파일(중간 세이브 파일 개념)을 사용해주세요.

밑바닥부터 다 돌려보고 싶다면 dataguide에서 데이터 받아서 돌리시면 됩니다만 권장하진 않습니다.

In [27]:
import yfinance as yf
import pandas as pd

# 수집할 지표의 티커(symbol) 목록
tickers = {
    'KRW_USD_Exchange': 'KRW=X', # 원/달러 환율
    # 추가적인 지표를 여기에 추가
}

# 데이터프레임을 저장할 딕셔너리
data = {}

# 각 티커에 대해 데이터 다운로드
for name, ticker in tickers.items():
    # 데이터 다운로드
    df = yf.download(ticker, start='2007-9-30', end='2024-12-01', interval='1mo')
    # 월말 데이터로 리샘플링
    df = df.resample('M').last()
    # 필요한 열만 선택
    df = df[['Adj Close']]
    # 열 이름 변경
    df.columns = [name]
    # 딕셔너리에 저장
    data[name] = df

# 모든 데이터를 하나의 데이터프레임으로 병합
result = pd.concat(data.values(), axis=1)

# 결과 출력
result.to_csv('KRW_USD_Exchange.csv')        

[*********************100%***********************]  1 of 1 completed


In [5]:
!pip install keras

In [1]:
import pandas as pd
import glob
from tqdm import tqdm
import os
import warnings
import gc
import numpy as np
import itertools
from tqdm import tqdm
import hotshot as hs
# import networkx as nx  # networkx 임포트
# from causalnex.structure.notears import from_pandas
# from causalnex.network import BayesianNetwork
# from econml.dr import DRLearner
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import zscore
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, GRU, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import plotly.graph_objs as go
from plotly.offline import iplot
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.decomposition import PCA
from imblearn.over_sampling import ADASYN
import warnings

warnings.filterwarnings('ignore')

# 팩터 전략별로 필요한 데이터를 계산하여 monthly_merged_df에 추가합니다.

if 'monthly_merged_df' not in globals():
    monthly_merged_df = pd.read_csv(
        'data/merged_df_monthly_preprocessing.csv',
        header=[0, 1, 2, 3],
        index_col=0,
        parse_dates=True
    )

pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

# CSV 부르기 및 기본적인 전처리 : 돌릴 필요 없음.

In [3]:
# DataFrame을 저장할 리스트 생성
df_list = []

# 1. 'data' 폴더 내에 'KSIF'가 포함된 CSV 파일 목록 가져오기
file_list = glob.glob('data/*KSIF*.csv')

# 파일이 존재하는지 확인
if not file_list:
    print("패턴에 맞는 파일을 찾을 수 없습니다.")
elif os.path.exists('data/merged_df_monthly.csv'):
    print("이미 통합한 월별 데이터 파일이 존재합니다. 해당 CSV를 불러옵니다.")
    merged_df_backup = pd.read_csv(
        'data/merged_df_monthly.csv',
        header=[0, 1, 2, 3],
        index_col=0,  # 첫 번째 열을 인덱스로 사용
        parse_dates=True  # 인덱스를 datetime으로 파싱
    )
    print("월별 CSV를 불러왔습니다.")
elif os.path.exists('data/merged_data.csv'):
    print("이미 통합한 일별 데이터 파일이 존재합니다. 해당 CSV를 월별로 전환합니다.")
    merged_df_backup = pd.read_csv(
        'data/merged_data.csv',
        header=[0, 1, 2, 3],
        index_col=0,
        parse_dates=True
    )
    # 인덱스를 datetime으로 변환
    merged_df_backup.index = pd.to_datetime(merged_df_backup.index, errors='coerce')
    
    # 월별 리샘플링
    merged_df_backup = merged_df_backup.resample('M').last()
    
    # 월별 데이터 저장
    merged_df_backup.to_csv('data/merged_df_monthly.csv', encoding='utf-8-sig')
    
    print("월별 리샘플링된 데이터가 'merged_df_monthly.csv'로 저장되었습니다.")
    print("월별 CSV를 불러왔습니다.")
else:
    # tqdm을 사용하여 진행 상황 표시
    for file_path in tqdm(file_list, desc="파일 처리 중"):
        # 각 CSV 파일 읽기 (적절한 인코딩과 인덱스 설정)
        try:
            df = pd.read_csv(
                file_path,
                skiprows=8,
                header=[0, 1, 2, 3, 4, 5],
                index_col=0,  # 첫 번째 열을 인덱스로 사용
                encoding='cp949',
                parse_dates=True
            )
        except UnicodeDecodeError:
            # 'cp949' 인코딩이 안 될 경우 'euc-kr'로 시도
            df = pd.read_csv(
                file_path,
                skiprows=8,
                header=[0, 1, 2, 3, 4, 5],
                index_col=0,
                encoding='euc-kr',
                parse_dates=True
            )
        except Exception as e:
            print(f"파일 {file_path}를 로드하는 중 에러 발생: {e}")
            continue  # 에러 발생 시 다음 파일로 넘어감
        
        # 멀티인덱스 컬럼에 이름 지정
        df.columns.names = ['Symbol', 'Symbol Name', 'Kind', 'item', 'item Name', 'Frequency']
        
        # 인덱스 이름 지정 ('Date'로 설정)
        df.index.name = 'Date'
        
        # 인덱스를 datetime으로 변환
        df.index = pd.to_datetime(df.index, errors='coerce')
        
        # 'Kind', 'Frequency' 레벨 제거하여 필요한 컬럼만 남김
        df.columns = df.columns.droplevel(['Kind', 'Frequency'])
        
        # 리스트에 DataFrame 추가
        df_list.append(df)
    
    # 2. 모든 DataFrame을 수평적으로 병합
    print("DataFrame 병합 중...")
    merged_df_backup = pd.concat(df_list, axis=1)
    del df_list  # 리스트 메모리에서 삭제
    gc.collect()  # 가비지 컬렉션 실행
    
    # 월별 리샘플링
    print("월별 리샘플링 중...")
    merged_df_backup = merged_df_backup.resample('M').last()
    
    # 월별 데이터 저장
    merged_df_backup.to_csv('data/merged_df_monthly.csv', encoding='utf-8-sig')
    
    print("월별 리샘플링된 데이터가 'merged_df_monthly.csv'로 저장되었습니다.")
    
    # 필요에 따라 일별 데이터를 저장하려면 아래 주석을 해제하세요.
    # merged_df_backup.to_csv('data/merged_data.csv', encoding='utf-8-sig')
    # print("모든 CSV 파일을 병합하여 'merged_data.csv'로 저장했습니다.")

    # 메모리 관리
    del merged_df_backup
    gc.collect()

이미 통합한 월별 데이터 파일이 존재합니다. 해당 CSV를 불러옵니다.
월별 CSV를 불러왔습니다.


In [2]:
def check_dataframe_issues(df_list):
    """
    점검 함수: 데이터프레임 리스트에서 고유하지 않은 인덱스와 기준 인덱스 불일치 확인
    Args:
        df_list (list): pandas 데이터프레임들의 리스트
    Returns:
        dict: 문제를 가진 데이터프레임의 인덱스 (non_unique_index, mismatched_index)
    """
    # 결과 저장용 딕셔너리
    issues = {"non_unique_index": [], "mismatched_index": []}
    
    # 기준 인덱스는 첫 번째 데이터프레임의 인덱스로 설정
    base_index = df_list[0].index

    # 각 데이터프레임 점검
    for i, df in enumerate(df_list):
        # 1. 고유하지 않은 인덱스 확인
        if not df.index.is_unique:
            issues["non_unique_index"].append(i)
        
        # 2. 기준 인덱스와 불일치 확인
        if not base_index.equals(df.index):
            issues["mismatched_index"].append(i)

    return issues

# 점검 실행
check_dataframe_issues(df_list)#여기서 인덱스 2번이 뜨는 이유는 얘가 하루 더 있거등요 ㅇㅇ

IndexError: list index out of range

In [3]:
merged_df_backup.columns.get_level_values(3).unique()#우리 데이터 뭐있나 함 볼까?

Index(['수정주가(원)', 'PER(보통)(배)', 'PER(직전4분기)(배)', 'PER(보통,자사주차감)(배)',
       'BPS(발표기준기말주식수)(원)', 'BPS(자사주차감)(원)', '상장주식수(주)', '시가총액 (평균)(원)',
       '상장주식수 (보통)(주)', '매출총이익(원)', '총자산(원)', '유동자산(원)', '현금및현금성자산(원)',
       '유동부채(원)', '단기차입금(원)', '이연법인세부채(원)', '거래대금(원)', '관리종목지정사유', '기타포괄손익(원)',
       '베타 (M,3Yr)', '베타 (D,1Yr)', '보통주자본금(원)', '수익률(%)', '수익률 (1개월)(%)',
       '수정주가 (52주 최고)(원)', '유무형자산상각비(원)', '이익잉여금(원)', 'Unnamed: 3182_level_4',
       '이익잉여금(천원)'],
      dtype='object', name='item Name')

In [ ]:
import numpy as np

# 월별 데이터로 작업할 merged_df 생성
merged_df = merged_df_backup.copy()

# 1. "(원)"으로 끝나는 컬럼 처리
# 'item Name'이 '(원)'으로 끝나는 컬럼 선택
won_mask = merged_df.columns.get_level_values('item Name').str.endswith('(원)')

# 쉼표 제거 및 숫자 변환을 벡터화된 연산으로 수행
# 문자열 'None', 'nan', '', 'N/A' 등을 NaN으로 변환
merged_df.loc[:, won_mask] = (
    merged_df.loc[:, won_mask]
    .astype(str)  # 모든 데이터를 문자열로 변환
    .replace(',', '', regex=True)  # 쉼표 제거
    .replace(['', 'None', 'nan', 'NaN', 'N/A'], np.nan)  # 비정상적인 값들을 NaN으로 변환
    .apply(pd.to_numeric, errors='coerce')  # 숫자로 변환 (변환 불가 시 NaN)
)

print("'(원)' 컬럼의 문자열 변환 및 숫자 변환 완료")

# 2. '홀딩스', '지주', '스펙'으로 끝나는 종목 제거
pattern = ('홀딩스', '지주', '스펙', '스팩')
symbol_names = merged_df.columns.get_level_values('Symbol Name')
mask = symbol_names.str.endswith(pattern)
merged_df = merged_df.loc[:, ~mask]

# 3. '관리종목지정사유' 처리
# '관리종목지정사유'가 있는 종목 추출
management_mask = merged_df.columns.get_level_values('item Name') == '관리종목지정사유'
management_df = merged_df.loc[:, management_mask]

# 인덱스를 datetime 형태로 변환
merged_df.index = pd.to_datetime(merged_df.index, errors='coerce')

# 각 종목별로 처리
for symbol in management_df.columns.get_level_values('Symbol').unique():
    symbol_management = management_df.loc[:, management_df.columns.get_level_values('Symbol') == symbol]
    
    # NaN이 아닌 첫 번째 날짜 찾기
    dates_with_issue = symbol_management[symbol_management.notna().any(axis=1)].index
    
    if not dates_with_issue.empty:
        try:
            # 이슈 발생 날짜
            issue_date = dates_with_issue[0]
            
            # 해당 Symbol의 데이터를 처리
            symbol_mask = merged_df.columns.get_level_values('Symbol') == symbol
            price_mask = merged_df.columns.get_level_values('item Name') == '수정주가(원)'
            other_mask = symbol_mask & ~price_mask
            
            # 이슈 발생 월부터 이후 데이터에 대해 NaN으로 설정 (수정주가는 제외)
            merged_df.loc[merged_df.index >= issue_date, other_mask] = np.nan
            
        except Exception as e:
            print(f"Error processing symbol: {symbol}, issue_date: {issue_date}, Error: {e}")

print('관리종목 포트폴리오 정상화')

# 4. '거래대금(원)' 기반 종목 제거
trading_value_mask = merged_df.columns.get_level_values('item Name') == '거래대금(원)'
trading_value_df = merged_df.loc[:, trading_value_mask]

# 인덱스를 datetime으로 변환
trading_value_df.index = pd.to_datetime(trading_value_df.index)

# 2014년 이후 데이터 선택
trading_value_df = trading_value_df[trading_value_df.index >= '2014-01-31']

# 문자열을 숫자로 변환 (오류 발생 시 NaN 처리)
trading_value_df = trading_value_df.apply(pd.to_numeric, errors='coerce')

# 거래대금이 4천만원 이하인 경우 True, NaN은 False로 처리
low_trading_value = (trading_value_df <= 40000000).fillna(False)

# 각 Symbol마다 거래대금이 4천만원 이하인 달이 하나라도 있는지 확인
symbols_to_remove = low_trading_value.any(axis=0)
symbols_to_remove = symbols_to_remove[symbols_to_remove].index.get_level_values('Symbol').unique().tolist()

# 해당 Symbol 제거
symbol_mask = merged_df.columns.get_level_values('Symbol').isin(symbols_to_remove)
merged_df = merged_df.loc[:, ~symbol_mask]

print('market impact 조정 완료')

# 5. 수정주가 기반 1개월 수익률 계산
# 인덱스를 datetime으로 변환
merged_df.index = pd.to_datetime(merged_df.index)

# '수정주가(원)' 데이터 추출
price_mask = merged_df.columns.get_level_values('item Name') == '수정주가(원)'
price_df = merged_df.loc[:, price_mask]

# 월별 수익률 계산
returns_df = price_df.pct_change()

# 'item Name'을 '1개월 수익률(계산)'으로 변경
returns_df.columns = pd.MultiIndex.from_tuples(
    [(symbol, symbol_name, item, '1개월 수익률(계산)') for symbol, symbol_name, item in zip(
        returns_df.columns.get_level_values('Symbol'),
        returns_df.columns.get_level_values('Symbol Name'),
        returns_df.columns.get_level_values('item')
    )],
    names=['Symbol', 'Symbol Name', 'item', 'item Name']
)

# 수익률 데이터를 merged_df에 추가
merged_df = pd.concat([merged_df, returns_df], axis=1)

print('1개월 수익률 계산 완료')

# 6. 결측치를 직전 값으로 대체
merged_df = merged_df.fillna(method='ffill')

print("전처리 완료")

In [ ]:
merged_df.to_csv('data/merged_df_monthly_preprocessing.csv', encoding='utf-8-sig')

In [22]:
merged_df_backup.dtypes

Symbol   Symbol Name  item        item Name       
A000660  SK하이닉스       S410000700  수정주가(원)               int64
                      6000701101  PER(보통)(배)          float64
                      6000701007  PER(직전4분기)(배)       float64
                      6000701006  PER(보통,자사주차감)(배)    float64
A373220  LG에너지솔루션     S410000700  수정주가(원)             float64
                                                       ...   
A900030  연합과기         S410000700  1개월 수익률(계산)         float64
A900060  중국식품포장       S410000700  1개월 수익률(계산)         float64
A900150  성융광전투자       S410000700  1개월 수익률(계산)         float64
A950030  네프로아이티       S410000700  1개월 수익률(계산)         float64
A950070  중국고섬         S410000700  1개월 수익률(계산)         float64
Length: 58059, dtype: object

# 팩터값 계산

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# 팩터 전략별로 필요한 데이터를 계산하여 monthly_merged_df에 추가합니다.

if 'monthly_merged_df' not in globals():
    monthly_merged_df = pd.read_csv(
        'data/merged_df_monthly_preprocessing.csv',
        header=[0, 1, 2, 3],
        index_col=0,
        parse_dates=True
    )

In [12]:
# ===== 팩터 전략 1: High P/E Ratio =====
# 상위 20% 종목에 롱 포지션, 하위 20% 종목에 숏 포지션을 취하는 전략

# 'PER(직전4분기)(배)' 데이터 추출
per_mask = monthly_merged_df.columns.get_level_values('item Name') == 'PER(보통,자사주차감)(배)'
per_df = monthly_merged_df.loc[:, per_mask]
per_df.columns = per_df.columns.droplevel(['item', 'item Name'])

# 결측치 처리 전에 per_df의 데이터를 float 타입으로 변환
per_df = per_df.apply(pd.to_numeric, errors='coerce')

# 결측치 처리
per_df = per_df.replace(0, np.nan)
per_df = per_df.replace(np.inf, np.nan)
per_df = per_df.fillna(method='ffill')

# 전체 종목에 대해 z-score 계산
per_zscore = -per_df.apply(lambda x: (x - x.mean()) / x.std(), axis=1)

# 팩터 값 저장
per_zscore.to_csv('factor_high_pe_ratio.csv', encoding='utf-8-sig')
print("'factor_high_pe_ratio.csv' 파일이 저장되었습니다.")

# 산업 분류 데이터 불러오기
try:
    industry_df = pd.read_csv(
        'data/industry.csv',
        header=[0, 1, 2, 3],
        index_col=0,
        parse_dates=True
    )
except UnicodeDecodeError:
    industry_df = pd.read_csv(
        'data/industry.csv',
        header=[0, 1, 2, 3],
        index_col=0,  # 첫 번째 열을 인덱스로 사용
        encoding='cp949',
        parse_dates=True  # 인덱스를 datetime으로 파싱
    )
    
# 멀티인덱스 설정
industry_df.columns.names = ['Symbol', 'Symbol Name', 'item', 'item Name']
industry_df.index.name = 'Date'

# '한국표준산업분류10차(대분류)', '한국표준산업분류10차(중분류)' 데이터 추출
industry_large_mask = industry_df.columns.get_level_values('item Name') == '한국표준산업분류11차(대분류)'
industry_medium_mask = industry_df.columns.get_level_values('item Name') == '한국표준산업분류11차(중분류)'

industry_large_df = industry_df.loc[:, industry_large_mask]
industry_large_df.columns = industry_large_df.columns.droplevel(['item', 'item Name'])
industry_medium_df = industry_df.loc[:, industry_medium_mask]
industry_medium_df.columns = industry_medium_df.columns.droplevel(['item', 'item Name'])

# PER 데이터와 산업 분류 데이터의 인덱스 및 컬럼 정렬
per_df, industry_large_df = per_df.align(industry_large_df, join='inner', axis=1)
per_df, industry_medium_df = per_df.align(industry_medium_df, join='inner', axis=1)

# 디버깅 출력을 위한 함수
def debug_print(message, df=None):
    print(message)
    if df is not None:
        print(df.head())
        print(df.shape)
        print("-" * 50)

# 산업별로 z-score 계산
def industry_zscore(per_series, industry_series):
    df = pd.DataFrame({'PER': per_series, 'Industry': industry_series})
    # NaN 값이 있는 행 제거
    df = df.dropna()
    # 산업별 그룹화 및 z-score 계산
    grouped = df.groupby('Industry')
    z_scores = grouped['PER'].transform(lambda x: -(x - x.mean()) / x.std() if x.std() != 0 else 0)
    return z_scores

# 대분류 산업별 z-score
per_zscore_large = per_df.copy()
print(per_zscore_large.shape)
for date in per_zscore_large.index:
    per_zscore_large.loc[date] = industry_zscore(per_df.loc[date], industry_large_df.loc[date])
    # 디버깅 출력
    # debug_print(f"[Large Industry] Date: {date}", per_zscore_large.loc[[date]])

# 멀티레벨 컬럼 구조 설정
per_zscore_large.columns = pd.MultiIndex.from_tuples(
    [(symbol, symbol_name, 'PER_zscore_large', 'PER_zscore_large') for symbol, symbol_name in per_zscore_large.columns],
    names=['Symbol', 'Symbol Name', 'item', 'item Name']
)

# 팩터 값 저장
per_zscore_large.to_csv('factor_high_pe_ratio_large_industry.csv', encoding='utf-8-sig')
print("'factor_high_pe_ratio_large_industry.csv' 파일이 저장되었습니다.")

# 중분류 산업별 z-score
per_zscore_medium = per_df.copy()
for date in per_zscore_medium.index:
    per_zscore_medium.loc[date] = industry_zscore(per_df.loc[date], industry_medium_df.loc[date])
    # 디버깅 출력
    # debug_print(f"[Medium Industry] Date: {date}", per_zscore_medium.loc[[date]])

# 멀티레벨 컬럼 구조 설정
per_zscore_medium.columns = pd.MultiIndex.from_tuples(
    [(symbol, symbol_name, 'PER_large', 'PER_large') for symbol, symbol_name in per_zscore_medium.columns],
    names=['Symbol', 'Symbol Name', 'item', 'item Name']
)
# 팩터 값 저장
per_zscore_medium.to_csv('factor_high_pe_ratio_medium_industry.csv', encoding='utf-8-sig')
print("'factor_high_pe_ratio_medium_industry.csv' 파일이 저장되었습니다.")

# 팩터 값 저장
per_zscore.to_csv('factor_high_pe_ratio.csv', encoding='utf-8-sig')

'factor_high_pe_ratio.csv' 파일이 저장되었습니다.
(206, 2052)
'factor_high_pe_ratio_large_industry.csv' 파일이 저장되었습니다.
'factor_high_pe_ratio_medium_industry.csv' 파일이 저장되었습니다.


In [7]:
# ===== 팩터 전략 2: HML (Kang, 2013) =====
# Book-to-Market Ratio 계산 (BPS / 주가)

# 'BPS(자사주차감)(원)' 데이터 추출
bps_mask = monthly_merged_df.columns.get_level_values('item Name') == 'BPS(자사주차감)(원)'
bps_df = monthly_merged_df.loc[:, bps_mask]

# '수정주가(원)' 데이터 추출
price_mask = monthly_merged_df.columns.get_level_values('item Name') == '수정주가(원)'
price_df = monthly_merged_df.loc[:, price_mask]

# 컬럼 레벨 중 'item'과 'item Name'을 제거하여 컬럼 정렬
bps_df.columns = bps_df.columns.droplevel(['item', 'item Name'])
price_df.columns = price_df.columns.droplevel(['item', 'item Name'])

# 인덱스 및 컬럼 정렬
bps_df, price_df = bps_df.align(price_df, join='inner', axis=0)
bps_df, price_df = bps_df.align(price_df, join='inner', axis=1)

# 계산 전에 데이터 타입 변환
bps_df = bps_df.apply(pd.to_numeric, errors='coerce')
price_df = price_df.apply(pd.to_numeric, errors='coerce')

# Book-to-Market Ratio 계산
bm_ratio_df = bps_df / price_df

# 결측치 처리
bm_ratio_df = bm_ratio_df.replace([np.inf, -np.inf], np.nan)
bm_ratio_df = bm_ratio_df.fillna(method='ffill')

# 멀티레벨 컬럼 구조 재설정
bm_ratio_df.columns = pd.MultiIndex.from_tuples(
    [(symbol, symbol_name, 'BM Ratio', 'BM Ratio') for symbol, symbol_name in bm_ratio_df.columns],
    names=['Symbol', 'Symbol Name', 'item', 'item Name']
)

# 팩터 값 저장
bm_ratio_df.to_csv('factor_hml.csv', encoding='utf-8-sig')
print("'factor_hml.csv' 파일이 저장되었습니다.")

'factor_hml.csv' 파일이 저장되었습니다.


In [8]:
# ===== 팩터 전략 5: Momentum 전략 =====
# 지난 12-1개월 수익률 계산 (직전 1개월은 제외)

# '수정주가(원)' 데이터 추출
price_mask = monthly_merged_df.columns.get_level_values('item Name') == '수정주가(원)'
price_df = monthly_merged_df.loc[:, price_mask]
price_df.columns = price_df.columns.droplevel(['item', 'item Name'])

# 데이터 타입 변환
price_df = price_df.apply(pd.to_numeric, errors='coerce')

# 인덱스 및 컬럼 정렬
price_df = price_df.sort_index()

# 결측치 처리
price_df = price_df.fillna(method='ffill')

# 12개월 전 가격과 1개월 전 가격 추출
price_12m_ago = price_df.shift(12)
price_1m_ago = price_df.shift(1)

# 모멘텀 계산
momentum_df = (price_1m_ago - price_12m_ago) / price_12m_ago

# 결측치 처리
momentum_df = momentum_df.replace([np.inf, -np.inf], np.nan)
momentum_df = momentum_df.fillna(method='ffill')

# 전체 종목에 대해 z-score 계산
momentum_zscore = momentum_df.apply(lambda x: (x - x.mean()) / x.std(), axis=1)

# 산업 분류 데이터 불러오기 (이미 불러온 industry_df 사용)
# '한국표준산업분류11차(대분류)', '한국표준산업분류11차(중분류)' 데이터 추출
industry_large_mask = industry_df.columns.get_level_values('item Name') == '한국표준산업분류11차(대분류)'
industry_medium_mask = industry_df.columns.get_level_values('item Name') == '한국표준산업분류11차(중분류)'

industry_large_df = industry_df.loc[:, industry_large_mask]
industry_large_df.columns = industry_large_df.columns.droplevel(['item', 'item Name'])
industry_medium_df = industry_df.loc[:, industry_medium_mask]
industry_medium_df.columns = industry_medium_df.columns.droplevel(['item', 'item Name'])

# 산업별로 z-score 계산
def industry_zscore(per_series, industry_series):
    df = pd.DataFrame({'PER': per_series, 'Industry': industry_series})
    # NaN 값이 있는 행 제거
    df = df.dropna()
    # 산업별 그룹화 및 z-score 계산
    grouped = df.groupby('Industry')
    z_scores = grouped['PER'].transform(lambda x: -(x - x.mean()) / x.std() if x.std() != 0 else 0)
    return z_scores


# 대분류 산업별 z-score
momentum_zscore_large = momentum_df.copy()
for date in momentum_zscore_large.index:
    momentum_zscore_large.loc[date] = industry_zscore(momentum_df.loc[date], industry_large_df.loc[date])

# 중분류 산업별 z-score
momentum_zscore_medium = momentum_df.copy()
for date in momentum_zscore_medium.index:
    momentum_zscore_medium.loc[date] = industry_zscore(momentum_df.loc[date], industry_medium_df.loc[date])

# 멀티레벨 컬럼 구조 설정
# 원래의 컬럼 정보를 사용하여 멀티인덱스 생성
momentum_df.columns = pd.MultiIndex.from_tuples(
    [(symbol, symbol_name, 'Momentum', 'Momentum') for symbol, symbol_name in momentum_df.columns],
    names=['Symbol', 'Symbol Name', 'item', 'item Name']
)

momentum_zscore.columns = pd.MultiIndex.from_tuples(
    [(symbol, symbol_name, 'Momentum_zscore', 'Momentum_zscore') for symbol, symbol_name in momentum_zscore.columns],
    names=['Symbol', 'Symbol Name', 'item', 'item Name']
)

momentum_zscore_large.columns = pd.MultiIndex.from_tuples(
    [(symbol, symbol_name, 'Momentum_large', 'Momentum_large') for symbol, symbol_name in momentum_zscore_large.columns],
    names=['Symbol', 'Symbol Name', 'item', 'item Name']
)

momentum_zscore_medium.columns = pd.MultiIndex.from_tuples(
    [(symbol, symbol_name, 'Momentum_medium', 'Momentum_medium') for symbol, symbol_name in momentum_zscore_medium.columns],
    names=['Symbol', 'Symbol Name', 'item', 'item Name']
)

# 팩터 값 저장
momentum_df.to_csv('factor_momentum.csv', encoding='utf-8-sig')
momentum_zscore.to_csv('factor_momentum_zscore.csv', encoding='utf-8-sig')
momentum_zscore_large.to_csv('factor_momentum_large_industry.csv', encoding='utf-8-sig')
momentum_zscore_medium.to_csv('factor_momentum_medium_industry.csv', encoding='utf-8-sig')

print("'factor_momentum.csv', 'factor_momentum_zscore.csv', 'factor_momentum_large_industry.csv', 'factor_momentum_medium_industry.csv' 파일이 저장되었습니다.")

'factor_momentum.csv', 'factor_momentum_zscore.csv', 'factor_momentum_large_industry.csv', 'factor_momentum_medium_industry.csv' 파일이 저장되었습니다.


In [9]:
# ===== 팩터 전략 6: Retained Earnings and Market-to-Book =====
# 이익잉여금(원) / 시가총액 계산

# '이익잉여금(원)' 데이터 추출
retained_earnings_mask = monthly_merged_df.columns.get_level_values('item Name') == '이익잉여금(원)'
retained_earnings_df = monthly_merged_df.loc[:, retained_earnings_mask]

# '시가총액 (평균)(원)' 데이터 추출
market_cap_mask = monthly_merged_df.columns.get_level_values('item Name') == '시가총액 (평균)(원)'
market_cap_df = monthly_merged_df.loc[:, market_cap_mask]

# 컬럼 레벨 중 'item'과 'item Name'을 제거하여 컬럼 정렬
retained_earnings_df.columns = retained_earnings_df.columns.droplevel(['item', 'item Name'])
market_cap_df.columns = market_cap_df.columns.droplevel(['item', 'item Name'])

# 인덱스 및 컬럼 정렬
retained_earnings_df, market_cap_df = retained_earnings_df.align(market_cap_df, join='inner', axis=0)
retained_earnings_df, market_cap_df = retained_earnings_df.align(market_cap_df, join='inner', axis=1)

# 데이터 타입 변환
retained_earnings_df = retained_earnings_df.apply(pd.to_numeric, errors='coerce')
market_cap_df = market_cap_df.apply(pd.to_numeric, errors='coerce')

# 이익잉여금 / 시가총액 계산
re_mc_ratio_df = retained_earnings_df / market_cap_df

# 결측치 처리
re_mc_ratio_df = re_mc_ratio_df.replace([np.inf, -np.inf], np.nan)
re_mc_ratio_df = re_mc_ratio_df.fillna(method='ffill')

# 멀티레벨 컬럼 구조 재설정
re_mc_ratio_df.columns = pd.MultiIndex.from_tuples(
    [(symbol, symbol_name, 'RE/MC Ratio', 'RE/MC Ratio') for symbol, symbol_name in re_mc_ratio_df.columns],
    names=['Symbol', 'Symbol Name', 'item', 'item Name']
)

# 팩터 값 저장
re_mc_ratio_df.to_csv('factor_retained_earnings.csv', encoding='utf-8-sig')
print("'factor_retained_earnings.csv' 파일이 저장되었습니다.")

'factor_retained_earnings.csv' 파일이 저장되었습니다.


In [10]:
# ===== 팩터 전략 10: Betting Against Beta =====
# 베타를 직접 계산하여 역수를 팩터 값으로 사용

import pandas as pd
import numpy as np
from tqdm import tqdm

# 1. 전처리된 월별 데이터에서 종목 리스트와 기간 추출
symbols = monthly_merged_df.columns.get_level_values('Symbol').unique()
dates = monthly_merged_df.index.unique()

# 각 종목별로 시작 날짜 추출
symbol_start_dates = {}
for symbol in symbols:
    # 해당 종목의 컬럼 선택
    symbol_cols = monthly_merged_df.loc[:, monthly_merged_df.columns.get_level_values('Symbol') == symbol]
    # 해당 종목의 데이터가 있는 날짜 추출
    symbol_data = symbol_cols.dropna(how='all')
    # 데이터가 있는 경우
    if not symbol_data.empty:
        start_date = symbol_data.index.min()
        # 시작 날짜에서 30일을 뺌
        adjusted_start_date = start_date - pd.Timedelta(days=30)
        # daily_df의 시작 날짜와 비교하여 실제 시작 날짜 결정
        symbol_start_dates[symbol] = adjusted_start_date
    else:
        # 데이터가 없는 경우 최소 날짜 설정
        symbol_start_dates[symbol] = pd.to_datetime('2000-01-01')  # 필요에 따라 최소 날짜 설정

print(f"전처리된 종목 수: {len(symbols)}")

# 2. 일별 데이터 로드 (data/KSIF_1.csv 파일)
file_path = 'data/KSIF_1.csv'

try:
    daily_df = pd.read_csv(
        file_path,
        skiprows=8,
        header=[0, 1, 2, 3, 4, 5],
        index_col=0,
        encoding='cp949',
        parse_dates=True
    )
except UnicodeDecodeError:
    daily_df = pd.read_csv(
        file_path,
        skiprows=8,
        header=[0, 1, 2, 3, 4, 5],
        index_col=0,
        encoding='euc-kr',
        parse_dates=True
    )
except Exception as e:
    print(f"파일 {file_path}를 로드하는 중 에러 발생: {e}")
    raise e  # 에러 발생 시 종료

# 멀티인덱스 컬럼 이름 지정
daily_df.columns.names = ['Symbol', 'Symbol Name', 'Kind', 'item', 'item Name', 'Frequency']
daily_df.index.name = 'Date'

# 'Kind', 'Frequency' 레벨 제거
daily_df.columns = daily_df.columns.droplevel(['Kind', 'Frequency'])

# 필요한 종목(Symbol)만 선택
symbol_mask = daily_df.columns.get_level_values('Symbol').isin(symbols)
daily_df = daily_df.loc[:, symbol_mask]

print(f"필터링된 종목 수: {len(daily_df.columns.get_level_values('Symbol').unique())}")

# '수정주가(원)' 데이터 추출
price_mask = daily_df.columns.get_level_values('item Name') == '수정주가(원)'
daily_price_df = daily_df.loc[:, price_mask]
daily_price_df.columns = daily_price_df.columns.droplevel(['item', 'item Name'])

# 데이터 정제: 쉼표 제거 및 숫자 변환
daily_price_df = (
    daily_price_df
    .astype(str)  # 모든 데이터를 문자열로 변환
    .replace(',', '', regex=True)  # 쉼표 제거
    .replace(['', 'None', 'nan', 'NaN', 'N/A', ''], np.nan)  # 비정상적인 값들을 NaN으로 변환
    .apply(pd.to_numeric, errors='coerce')  # 숫자로 변환 (변환 불가 시 NaN)
)

# 데이터 타입 확인
print("일별 가격 데이터 타입 확인:")
print(daily_price_df.dtypes.unique())

# 종목별 일별 수익률 계산
daily_returns_dict = {}
for symbol in symbols:
    if symbol in daily_price_df.columns:
        symbol_price = daily_price_df[symbol]
        if not symbol_price.empty:
            # 해당 종목의 시작 날짜 계산
            start_date = symbol_start_dates[symbol]
            # 시작 날짜부터 데이터 선택
            symbol_price = symbol_price.loc[start_date:]
            # 수익률 계산
            symbol_returns = symbol_price.pct_change().dropna()
            daily_returns_dict[symbol] = symbol_returns
        else:
            # 해당 종목의 데이터가 없는 경우
            daily_returns_dict[symbol] = pd.Series(dtype=float)
    else:
        daily_returns_dict[symbol] = pd.Series(dtype=float)

print(f"일별 수익률 계산 완료. 종목 수: {len(daily_returns_dict)}")

# 시장 수익률 계산 (종가지수(포인트) 기반)
try:
    market_df = pd.read_csv(
        'data/kor_market.csv',
        skiprows=8,
        header=[0, 1, 2, 3, 4, 5],
        index_col=0,
        encoding='cp949',
        parse_dates=True
    )
except UnicodeDecodeError:
    market_df = pd.read_csv(
        'data/kor_market.csv',
        skiprows=8,
        header=[0, 1, 2, 3, 4, 5],
        index_col=0,
        encoding='euc-kr',
        parse_dates=True
    )

# 멀티인덱스 컬럼 이름 지정
market_df.columns.names = ['Symbol', 'Symbol Name', 'Kind', 'item', 'item Name', 'Frequency']
market_df.index.name = 'Date'

# 'Kind', 'Frequency' 레벨 제거
market_df.columns = market_df.columns.droplevel(['Kind', 'Frequency'])

# '종가지수(포인트)' 데이터 추출
index_mask = market_df.columns.get_level_values('item Name') == '종가지수(포인트)'
index_df = market_df.loc[:, index_mask]

# '코스피'와 '코스닥' 지수만 선택
symbol_names = index_df.columns.get_level_values('Symbol Name')
kospi_kosdaq_mask = (symbol_names == '코스피') | (symbol_names == '코스닥')
kospi_kosdaq_indices = index_df.loc[:, kospi_kosdaq_mask]
kospi_kosdaq_indices.columns = kospi_kosdaq_indices.columns.droplevel(['item', 'item Name'])

# 데이터 정제: 쉼표 제거 및 숫자 변환
kospi_kosdaq_indices = (
    kospi_kosdaq_indices
    .astype(str)
    .replace(',', '', regex=True)
    .replace(['', 'None', 'nan', 'NaN', 'N/A', ''], np.nan)
    .apply(pd.to_numeric, errors='coerce')
)

# 코스피와 코스닥 지수의 일별 수익률 계산
market_returns = kospi_kosdaq_indices.mean(axis=1).pct_change().dropna()

# 시장 수익률 데이터 기간 확인
print(f"시장 수익률 데이터 기간: {market_returns.index.min()} ~ {market_returns.index.max()}")

# 3. 베타 계산 함수 정의 및 계산
def calculate_beta(stock_returns, market_returns, window=365):
    # 결측치 제거
    combined = pd.concat([stock_returns, market_returns], axis=1).dropna()
    if len(combined) < 30:
        return np.nan
    else:
        stock_ret = combined.iloc[:, 0]
        market_ret = combined.iloc[:, 1]
        cov = stock_ret.cov(market_ret)
        var = market_ret.var()
        beta = cov / var if var != 0 else np.nan
        return beta

# 베타 값을 저장할 데이터프레임 생성
beta_df = pd.DataFrame(index=dates, columns=symbols)

for date in tqdm(dates, desc='베타 계산 중'):
    for symbol in symbols:
        # 해당 종목의 일별 수익률 시리즈
        stock_returns = daily_returns_dict[symbol]
        # 해당 날짜까지의 데이터 사용
        stock_returns = stock_returns[stock_returns.index <= date]
        market_returns_up_to_date = market_returns[market_returns.index <= date]
        # 최근 window 기간의 데이터 추출
        stock_returns = stock_returns.iloc[-365:]
        market_returns_up_to_date = market_returns_up_to_date.iloc[-365:]
        # 베타 계산
        beta = calculate_beta(stock_returns, market_returns_up_to_date)
        beta_df.at[date, symbol] = beta

# 베타의 역수를 팩터 값으로 사용
inv_beta_df = (1 / beta_df.astype(float)).replace([np.inf, -np.inf], np.nan)

# 멀티레벨 컬럼 구조 설정
inv_beta_df.columns = pd.MultiIndex.from_tuples(
    [(symbol, monthly_merged_df.columns.get_level_values('Symbol Name')[monthly_merged_df.columns.get_level_values('Symbol') == symbol][0], 'Inverse Beta', 'Inverse Beta') for symbol in inv_beta_df.columns],
    names=['Symbol', 'Symbol Name', 'item', 'item Name']
)

inv_beta_df.index.name = 'Date'

# 팩터 값 저장
inv_beta_df.to_csv('factor_betting_against_beta.csv', encoding='utf-8-sig')

print("'factor_betting_against_beta.csv' 파일이 저장되었습니다.")

전처리된 종목 수: 2053
필터링된 종목 수: 2052
일별 가격 데이터 타입 확인:
[dtype('int64') dtype('float64')]
일별 수익률 계산 완료. 종목 수: 2053
시장 수익률 데이터 기간: 2007-10-21 00:00:00 ~ 2024-11-20 00:00:00


베타 계산 중: 100%|██████████████████████████████████████████████████████████████████| 206/206 [09:47<00:00,  2.85s/it]


'factor_betting_against_beta.csv' 파일이 저장되었습니다.


In [5]:
#계산한 팩터값에 결측치 확인
import pandas as pd

# ===== 팩터 값 CSV 확인 및 결측치 분석 =====

# 저장된 팩터 값 CSV 파일 목록
factor_files = [
    'factor_high_pe_ratio.csv',
    'factor_high_pe_ratio_large_industry.csv',
    'factor_high_pe_ratio_medium_industry.csv',
    'factor_hml.csv',
    'factor_momentum.csv',
    'factor_retained_earnings.csv',
    'factor_betting_against_beta.csv'
]

# 결과를 저장할 리스트
missing_data_summary = []

for file in factor_files:
    print(f"Processing file: {file}")
    try:
        # CSV 파일 로드
        factor_df = pd.read_csv(file, 
        header=[0, 1, 2, 3],
        index_col=0,
        parse_dates=True
        )

        # 멀티인덱스 설정
        factor_df.columns.names = ['Symbol', 'Symbol Name', 'item', 'item Name']
        factor_df.index.name = 'Date'
        
        print(f"  Loaded successfully. Shape: {factor_df.shape}")
        
        # 2008년 이후 데이터만 필터링
        factor_df = factor_df.loc[factor_df.index >= '2008-01-01']
        
        # 결측치 분석
        missing_summary = factor_df.isna().sum(axis=1)  # 각 날짜별 결측치 수
        total_columns = factor_df.shape[1]  # 전체 컬럼 수
        
        # 가장 결측치가 많은 날짜와 해당 날짜의 결측치 비율
        max_missing_date = missing_summary.idxmax()
        max_missing_count = missing_summary.max()
        max_missing_ratio = (max_missing_count / total_columns) * 100  # 결측치 비율
        
        # 결측치 요약 추가
        missing_data_summary.append({
            'Factor File': file,
            'Max Missing Date': max_missing_date,
            'Max Missing Count': max_missing_count,
            'Total Columns': total_columns,
            'Max Missing Ratio (%)': max_missing_ratio
        })
        
        print(f"  Max Missing Date: {max_missing_date}")
        print(f"  Max Missing Count: {max_missing_count}")
        print(f"  Total Columns: {total_columns}")
        print(f"  Max Missing Ratio (%): {max_missing_ratio:.2f}%")
        
    except Exception as e:
        print(f"  Error processing file {file}: {e}")

# 결측치 분석 결과 DataFrame 생성
missing_summary_df = pd.DataFrame(missing_data_summary)

# 결측치 분석 결과 출력
print("\n===== Missing Data Summary =====")
print(missing_summary_df)

# 결측치 분석 결과 저장
missing_summary_df.to_csv('factor_missing_data_summary.csv', index=False, encoding='utf-8-sig')
print("\n결측치 분석 결과가 'factor_missing_data_summary.csv' 파일로 저장되었습니다.")

Processing file: factor_high_pe_ratio.csv
  Loaded successfully. Shape: (206, 2067)
  Max Missing Date: 2008-01-31 00:00:00
  Max Missing Count: 1544
  Total Columns: 2067
  Max Missing Ratio (%): 74.70%
Processing file: factor_high_pe_ratio_large_industry.csv
  Error processing file factor_high_pe_ratio_large_industry.csv: Length of new names must be 1, got 4
Processing file: factor_high_pe_ratio_medium_industry.csv
  Error processing file factor_high_pe_ratio_medium_industry.csv: Length of new names must be 1, got 4
Processing file: factor_hml.csv
  Error processing file factor_hml.csv: Length of new names must be 1, got 4
Processing file: factor_momentum.csv
  Error processing file factor_momentum.csv: Length of new names must be 1, got 4
Processing file: factor_retained_earnings.csv
  Error processing file factor_retained_earnings.csv: Length of new names must be 1, got 4
Processing file: factor_betting_against_beta.csv
  Loaded successfully. Shape: (426005, 5)
  Max Missing Date: 

# 백테스팅

In [140]:
def backtest_strategy(factor_csv, merged_df, rebalancing_period=1, long_only=True, threshold=0.2, cutoff=0.0, reversal=False, weighting_method='equal', start_date='2008-10-31'):
    """
    백테스팅 함수를 구현합니다.

    Parameters:
    - factor_csv (str): 팩터값 CSV 파일의 경로
    - merged_df (pd.DataFrame): 수익률 데이터가 포함된 데이터프레임
    - rebalancing_period (int): 리밸런싱 주기 (1, 3, 6, 12 중 하나)
    - long_only (bool): 롱 온리 전략 여부
    - threshold (float): 포지션을 취할 상위/하위 퍼센트 (0 < threshold <= 1)
    - cutoff (float): 포지션을 취할 시작 퍼센트 (0 <= cutoff < threshold)
    - reversal (bool): 전략을 반대로 적용할지 여부
    - weighting_method (str): 'equal' 또는 'value' 중 하나로, 동일가중 또는 가치가중을 결정
    - start_date (str): 백테스트 시작 날짜 (예: '2008-10-31')

    Returns:
    - results_df (pd.DataFrame): 월별 포트폴리오 변동과 포지션을 포함한 데이터프레임
    """
    # 팩터 데이터 불러오기
    factor_df = pd.read_csv(factor_csv, index_col=0, header=[0, 1, 2, 3], parse_dates=True)
    factor_df.index.name = 'Date'
    factor_df.columns.names = ['Symbol', 'Symbol Name', 'item', 'item Name']

    # 팩터 데이터에서 'item'과 'item Name' 레벨을 제거하여 심볼과 종목명만 남김
    factor_df.columns = factor_df.columns.droplevel(['item', 'item Name'])

    # 팩터 데이터 날짜 필터링 (백테스트 시작일 이후 데이터만 사용)
    factor_df = factor_df[factor_df.index >= start_date]

    # 수익률 데이터 추출 ('1개월 수익률(계산)')
    returns_mask = merged_df.columns.get_level_values('item Name') == '1개월 수익률(계산)'
    returns_df = merged_df.loc[:, returns_mask]
    returns_df.columns = returns_df.columns.droplevel(['item', 'item Name'])
    returns_df.columns.names = ['Symbol', 'Symbol Name']

    # 수익률 데이터 날짜 필터링 (백테스트 시작일 이후 데이터만 사용)
    returns_df = returns_df[returns_df.index >= start_date]

    # 월말 기준으로 데이터 리샘플링
    factor_df = factor_df.resample('M').last()
    returns_df = returns_df.resample('M').last()

    # 팩터 데이터와 수익률 데이터의 공통 심볼(Symbol)만 사용
    common_symbols = factor_df.columns.intersection(returns_df.columns)
    factor_df = factor_df[common_symbols]
    returns_df = returns_df[common_symbols]

    # 팩터 데이터와 수익률 데이터를 날짜와 심볼 기준으로 정렬
    factor_df = factor_df.sort_index().sort_index(axis=1)
    returns_df = returns_df.sort_index().sort_index(axis=1)

    # 리밸런싱 날짜 설정
    rebalancing_dates = factor_df.index[::rebalancing_period]

    # 포트폴리오 초기화
    portfolio = pd.DataFrame(index=returns_df.index, columns=['Portfolio Value', 'Monthly Return', 'Transaction Cost'])
    portfolio['Portfolio Value'] = 1.0  # 초기 투자금 1로 설정
    portfolio['Transaction Cost'] = 0.0  # 초기 거래 비용 0으로 설정

    # 각 날짜의 포지션을 저장할 딕셔너리
    positions = {}

    # 거래 비용 비율 설정 (0.1%)
    transaction_cost_rate = 0.001

    # 백테스트 진행
    for i, date in enumerate(returns_df.index):
        # 이전 날짜 설정
        if i > 0:
            prev_date = returns_df.index[i - 1]
        else:
            prev_date = None

        # 리밸런싱 시점인지 확인
        if date in rebalancing_dates:
            # 리밸런싱 시점에서는 새로운 포지션을 설정
            factor = factor_df.loc[date].dropna()

            # 종목 수 계산
            num_assets = len(factor)
            num_selected = int(num_assets * threshold)
            num_cutoff = int(num_assets * cutoff)

            if long_only:
                # 롱 온리 전략
                if reversal:
                    # 하위 cutoff% ~ threshold% 종목을 선택
                    selected_symbols = factor.nsmallest(num_selected + num_cutoff).iloc[num_cutoff:].index
                else:
                    # 상위 cutoff% ~ threshold% 종목을 선택
                    selected_symbols = factor.nlargest(num_selected + num_cutoff).iloc[num_cutoff:].index

                # 가중치 계산
                if weighting_method == 'equal':
                    weights = pd.Series(1.0 / len(selected_symbols), index=selected_symbols)
                elif weighting_method == 'value':
                    weights = factor[selected_symbols] / factor[selected_symbols].abs().sum()
                else:
                    raise ValueError("weighting_method must be 'equal' or 'value'")
            else:
                # 롱숏 전략
                if reversal:
                    # 하위 cutoff% ~ threshold% 종목을 롱, 상위 cutoff% ~ threshold% 종목을 숏
                    long_symbols = factor.nsmallest(num_selected + num_cutoff).iloc[num_cutoff:].index
                    short_symbols = factor.nlargest(num_selected + num_cutoff).iloc[num_cutoff:].index
                else:
                    # 상위 cutoff% ~ threshold% 종목을 롱, 하위 cutoff% ~ threshold% 종목을 숏
                    long_symbols = factor.nlargest(num_selected + num_cutoff).iloc[num_cutoff:].index
                    short_symbols = factor.nsmallest(num_selected + num_cutoff).iloc[num_cutoff:].index

                # 가중치 계산
                if weighting_method == 'equal':
                    long_weights = pd.Series(1.0 / len(long_symbols), index=long_symbols)
                    short_weights = pd.Series(-1.0 / len(short_symbols), index=short_symbols)
                elif weighting_method == 'value':
                    long_weights = factor[long_symbols] / factor[long_symbols].abs().sum()
                    short_weights = -factor[short_symbols] / factor[short_symbols].abs().sum()
                else:
                    raise ValueError("weighting_method must be 'equal' or 'value'")

                # 롱과 숏 포지션 합치기
                weights = pd.concat([long_weights, short_weights])

            # 거래 비용 계산 (포지션 변경에 따른)
            if prev_date is not None and prev_date in positions:
                prev_weights = positions[prev_date]
                # 포지션 변경량 계산
                weight_diff = weights.reindex(prev_weights.index).fillna(0) - prev_weights.reindex(weights.index).fillna(0)
            else:
                # 처음 포지션을 잡을 때는 전체 포지션이 신규 진입
                weight_diff = weights

            # 거래 비용 계산
            transaction_cost = transaction_cost_rate * weight_diff.abs().sum()

            # 현재 포지션 저장
            positions[date] = weights

        else:
            # 리밸런싱 시점이 아니면 이전 포지션을 그대로 유지
            if prev_date in positions:
                positions[date] = positions[prev_date]
                transaction_cost = 0.0  # 거래 비용 없음
            else:
                # 이전 포지션이 없으면 포지션 없음
                positions[date] = pd.Series()
                transaction_cost = 0.0  # 거래 비용 없음

        # 수익률 계산 (선행 편향 방지를 위해 이전 포지션 사용)
        if prev_date in positions and not positions[prev_date].empty:
            weights = positions[prev_date]

            # 해당 월의 수익률 계산
            returns = returns_df.loc[date]
            # 포지션에 해당하는 종목의 수익률만 선택
            aligned_returns = returns.reindex(weights.index).fillna(0)
            # 포트폴리오 수익률 계산 (벡터화 연산)
            portfolio_return = (weights * aligned_returns).sum()
        else:
            # 포지션이 없으면 수익률 0
            portfolio_return = 0.0

        # 거래 비용을 수익률에 반영
        net_return = portfolio_return - transaction_cost

        # 포트폴리오 가치 업데이트
        if i > 0:
            portfolio.loc[date, 'Monthly Return'] = net_return
            portfolio.loc[date, 'Transaction Cost'] = transaction_cost
            portfolio.loc[date, 'Portfolio Value'] = portfolio.iloc[i - 1]['Portfolio Value'] * (1 + net_return)
        else:
            # 첫 번째 기간은 수익률 계산하지 않음
            portfolio.loc[date, 'Monthly Return'] = 0.0
            portfolio.loc[date, 'Transaction Cost'] = 0.0

    # 포지션 정보를 데이터프레임으로 변환
    positions_df = pd.DataFrame.from_dict(positions, orient='index')
    positions_df.index.name = 'Date'

    # 결과 합치기
    results_df = portfolio.join(positions_df, how='left')

    return results_df

In [141]:
# 'factor'라는 글자가 포함된 모든 CSV 파일 목록을 가져옵니다.
factor_files = glob.glob('*factor*.csv')

# 백테스팅에 사용할 파라미터 그리드 설정
rebalancing_periods = [1, 3, 6]        # 리밸런싱 주기
thresholds = [0.1, 0.2, 0.3]           # 상위/하위 퍼센트
cutoffs = [0, 0.05, 0.1]               # 시작 퍼센트
weighting_methods = ['equal', 'value'] # 가중치 방법
start_date = '2008-10-31'              # 백테스트 시작일

# 모든 팩터 파일에 대해 백테스팅 수행
for factor_csv in tqdm(factor_files):
    # print(f"백테스팅 시작: {factor_csv}")
    
    # 팩터 파일명에서 확장자를 제거하여 폴더명을 생성합니다.
    factor_name = os.path.splitext(factor_csv)[0]
    
    # 팩터별 폴더 생성 (이미 존재하면 생략)
    if not os.path.exists(factor_name):
        os.makedirs(factor_name)
    
    # 모든 파라미터 조합에 대해 백테스팅 수행
    for rebalancing_period, threshold, cutoff, weighting_method in itertools.product(
        rebalancing_periods, thresholds, cutoffs, weighting_methods):
        
        # cutoff는 threshold보다 작아야 합니다.
        if cutoff >= threshold:
            continue
        
        # 백테스팅 함수 호출
        try:
            results_df = backtest_strategy(
                factor_csv=factor_csv,
                merged_df=monthly_merged_df,  # 수익률 데이터가 포함된 데이터프레임 (사전에 정의되어 있어야 합니다)
                rebalancing_period=rebalancing_period,
                long_only=True,  # Long-only 전략
                threshold=threshold,
                cutoff=cutoff,
                reversal=False,  # 기본값
                weighting_method=weighting_method,
                start_date=start_date
            )
            
            # 파일명 생성 (스네이크 케이스로 파라미터 명명)
            file_name = f"rebalancing_{rebalancing_period}_threshold_{threshold}_cutoff_{cutoff}_weighting_{weighting_method}"
            file_name = file_name.replace('.', '_')  # 파일명에 있는 점을 언더스코어로 변경
            
            # 결과 저장 경로 생성
            save_path = os.path.join(factor_name, file_name)
            
            # 백테스팅 결과를 CSV 파일로 저장
            results_df.to_csv(save_path+'.csv', encoding='utf-8-sig')
            
            # print(f"완료: {factor_csv}, 리밸런싱 주기: {rebalancing_period}, threshold: {threshold}, cutoff: {cutoff}, 가중치 방법: {weighting_method}")
            # print(f"저장 위치: {save_path}")
        except Exception as e:
            print(f"에러 발생: {e}")
            continue

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [17:17<00:00, 103.75s/it]


In [142]:
import glob
import os
import pandas as pd
import numpy as np

# 시장 수익률 계산 (종가지수(포인트) 기반)
def get_risk_free_rate():
    """
    위험 무시 수익률(rf)을 계산하는 함수입니다.
    코스피와 코스닥 지수의 일별 수익률을 평균하여 사용합니다.
    월별 수익률로 변환하여 반환합니다.
    
    Returns:
    - rf_monthly_returns (pd.Series): 월별 위험 무시 수익률 시리즈
    """
    # 시장 데이터 로드
    try:
        market_df = pd.read_csv(
            'data/rf.csv',
            skiprows=8,
            header=[0, 1, 2, 3, 4, 5],
            index_col=0,
            encoding='cp949',
            parse_dates=True
        )
    except UnicodeDecodeError:
        market_df = pd.read_csv(
            'data/rf.csv',
            skiprows=8,
            header=[0, 1, 2, 3, 4, 5],
            index_col=0,
            encoding='euc-kr',
            parse_dates=True
        )

    # 멀티인덱스 컬럼 이름 지정
    market_df.columns.names = ['Symbol', 'Symbol Name', 'Kind', 'item', 'item Name', 'Frequency']
    market_df.index.name = 'Date'

    # 'Kind', 'Frequency' 레벨 제거
    market_df.columns = market_df.columns.droplevel(['Kind', 'Frequency'])

    # '종가지수(포인트)' 데이터 추출
    # index_mask = market_df.columns.get_level_values('item Name') == '종가지수(포인트)'
    index_mask = market_df.columns.get_level_values('item Name') == '시장금리:국고1년(%)'
    index_df = market_df.loc[:, index_mask]

    # '코스피'와 '코스닥' 지수만 선택
    symbol_names = index_df.columns.get_level_values('Symbol Name')
    # kospi_kosdaq_mask = (symbol_names == '코스피') | (symbol_names == '코스닥')
    kospi_kosdaq_mask = (symbol_names == 'ECO')
    kospi_kosdaq_indices = index_df.loc[:, kospi_kosdaq_mask]
    kospi_kosdaq_indices.columns = kospi_kosdaq_indices.columns.droplevel(['item', 'item Name'])

    # 데이터 정제: 쉼표 제거 및 숫자 변환
    kospi_kosdaq_indices = (
        kospi_kosdaq_indices
        .astype(str)
        .replace(',', '', regex=True)
        .replace(['', 'None', 'nan', 'NaN', 'N/A', ''], np.nan)
        .apply(pd.to_numeric, errors='coerce')
    )

    # 코스피와 코스닥 지수의 일별 수익률 계산
    market_returns = kospi_kosdaq_indices.mean(axis=1).pct_change().dropna()

    # 월별 수익률로 변환
    rf_monthly_returns = market_returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
    rf_monthly_returns.name = 'Risk-Free Rate'

    return rf_monthly_returns

# 위험 무시 수익률(rf) 시리즈 가져오기
rf_returns = get_risk_free_rate()

# Sharpe Ratio를 계산하는 함수 정의
def calculate_sharpe_ratio(returns_series, rf_series, start_date, end_date):
    """
    Sharpe Ratio를 계산하는 함수입니다.
    
    Parameters:
    - returns_series (pd.Series): 월별 수익률 시리즈
    - rf_series (pd.Series): 월별 위험 무시 수익률 시리즈
    - start_date (str): Sharpe Ratio 계산에 사용할 시작 날짜 (예: '2008-10-31')
    - end_date (str): Sharpe Ratio 계산에 사용할 종료 날짜 (예: '2018-09-30')
    
    Returns:
    - sharpe_ratio (float): Sharpe Ratio 값
    """
    # 지정된 기간으로 데이터 필터링
    returns_series = returns_series.loc[start_date:end_date]
    rf_series = rf_series.loc[start_date:end_date]

    # 수익률과 위험 무시 수익률의 교집합 날짜 선택
    common_index = returns_series.index.intersection(rf_series.index)
    returns = returns_series.loc[common_index].dropna()
    rf = rf_series.loc[common_index].dropna()

    # 위험 프리미엄 계산 (초과 수익률)
    excess_returns = returns - rf

    # 결측치 제거
    excess_returns = excess_returns.dropna()

    # 월별 초과 수익률의 평균과 표준편차 계산
    mean_excess_return = excess_returns.mean()
    std_excess_return = excess_returns.std()

    # Sharpe Ratio 계산 (연율화)
    if std_excess_return != 0:
        sharpe_ratio = (mean_excess_return / std_excess_return) * np.sqrt(12)
    else:
        sharpe_ratio = np.nan
    return sharpe_ratio

# 'factor'로 시작하는 모든 폴더 목록을 가져옵니다.
factor_folders = [folder for folder in glob.glob('factor*') if os.path.isdir(folder)]

# 각 팩터별로 Sharpe Ratio가 가장 높은 파일명을 저장할 리스트 초기화
best_results = []

# Training 데이터 기간 설정
training_start_date = '2008-10-31'
training_end_date = '2018-09-30'

# 각 팩터 폴더에 대해 반복
for folder in tqdm(factor_folders):
    # print(f"폴더 처리 중: {folder}")
    # 폴더 내의 모든 CSV 파일 목록 가져오기
    csv_files = glob.glob(os.path.join(folder, '*.csv'))
    
    # Sharpe Ratio를 저장할 딕셔너리 초기화
    sharpe_ratios = {}
    
    # 각 CSV 파일에 대해 반복
    for csv_file in csv_files:
        try:
            # CSV 파일 읽기
            df = pd.read_csv(csv_file, index_col=0, parse_dates=True)
            # 'Monthly Return' 컬럼이 존재하는지 확인
            if 'Monthly Return' in df.columns:
                # Sharpe Ratio 계산
                sharpe_ratio = calculate_sharpe_ratio(df['Monthly Return'], rf_returns, training_start_date, training_end_date)
                # Sharpe Ratio 저장
                sharpe_ratios[csv_file] = sharpe_ratio
            else:
                print(f"'Monthly Return' 컬럼이 없습니다: {csv_file}")
        except Exception as e:
            print(f"에러 발생: {csv_file}, 에러 내용: {e}")
            continue
    
    # 해당 팩터 폴더에서 Sharpe Ratio가 가장 높은 파일 찾기
    if sharpe_ratios:
        # 최대 Sharpe Ratio를 가진 파일 찾기
        best_file = max(sharpe_ratios, key=sharpe_ratios.get)
        best_sharpe = sharpe_ratios[best_file]
        # print(f"최고 Sharpe Ratio: {best_sharpe:.4f}, 파일명: {best_file}")
        # 파일명을 리스트에 추가
        best_results.append(best_file)
    else:
        print(f"Sharpe Ratio를 계산할 수 있는 파일이 없습니다: {folder}")

# best_results 리스트를 'best_results.txt' 파일에 저장
with open('best_results.txt', 'w', encoding='utf-8') as f:
    for file_path in best_results:
        f.write(file_path + '\n')

# 결과 출력
print("\n각 팩터별로 Sharpe Ratio가 가장 높은 파일 목록:")
for best_file in best_results:
    print(best_file)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:18<00:00,  1.87s/it]


각 팩터별로 Sharpe Ratio가 가장 높은 파일 목록:
factor_betting_against_beta\rebalancing_1_threshold_0_3_cutoff_0_1_weighting_equal.csv
factor_high_pe_ratio\rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal.csv
factor_high_pe_ratio_large_industry\rebalancing_3_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_high_pe_ratio_medium_industry\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_value.csv
factor_hml\rebalancing_3_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_momentum\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_momentum_large_industry\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_momentum_medium_industry\rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal.csv
factor_momentum_zscore\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_retained_earnings\rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal.csv


# 최종 선택된 경주마 발표(시각화)

factor_betting_against_beta\rebalancing_1_threshold_0_3_cutoff_0_1_weighting_equal.csv
factor_high_pe_ratio\rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal.csv
factor_high_pe_ratio_large_industry\rebalancing_3_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_high_pe_ratio_medium_industry\rebalancing_1_threshold_0_2_cutoff_0_05_weighting_equal.csv
factor_hml\rebalancing_1_threshold_0_3_cutoff_0_1_weighting_equal.csv
factor_momentum\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_momentum_large_industry\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_momentum_medium_industry\rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal.csv
factor_momentum_zscore\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_retained_earnings\rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal.csv

In [143]:
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio
import os

# Plotly 렌더러 설정: 브라우저에서 그래프를 표시하도록 설정
pio.renderers.default = 'browser'

# Training 데이터 기간 설정
training_start_date = '2008-10-31'
training_end_date = '2018-09-30'

# 그래프의 트레이스를 저장할 리스트 초기화
traces = []

# Sharpe Ratio를 저장할 리스트 초기화
sharpe_ratio_list = []

# Sharpe Ratio 계산 함수 (이미 정의되어 있다고 가정)
def calculate_sharpe_ratio(returns_series, rf_series, start_date, end_date):
    # 지정된 기간으로 데이터 필터링
    returns_series = returns_series.loc[start_date:end_date]
    rf_series = rf_series.loc[start_date:end_date]

    # 수익률과 위험 무시 수익률의 교집합 날짜 선택
    common_index = returns_series.index.intersection(rf_series.index)
    returns = returns_series.loc[common_index].dropna()
    rf = rf_series.loc[common_index].dropna()

    # 위험 프리미엄 계산 (초과 수익률)
    excess_returns = returns - rf

    # 결측치 제거
    excess_returns = excess_returns.dropna()

    # 월별 초과 수익률의 평균과 표준편차 계산
    mean_excess_return = excess_returns.mean()
    std_excess_return = excess_returns.std()

    # Sharpe Ratio 계산 (연율화)
    if std_excess_return != 0:
        sharpe_ratio = (mean_excess_return / std_excess_return) * np.sqrt(12)
    else:
        sharpe_ratio = np.nan
    return sharpe_ratio

# 위험 무시 수익률(rf) 시리즈 가져오기 (이미 정의되어 있다고 가정)
# rf_returns = get_risk_free_rate()

# best_results 리스트에 담긴 CSV 파일들을 순회하면서 차트 생성 및 Sharpe Ratio 계산
for csv_file in best_results:
    # CSV 파일 읽기
    df = pd.read_csv(csv_file, index_col=0, parse_dates=True)

    # 기간 필터링 (Sharpe Ratio 계산 기간과 동일)
    df_period = df.loc[training_start_date:training_end_date]

    # 포트폴리오 가치와 월별 수익률이 존재하는지 확인
    if 'Portfolio Value' in df_period.columns and 'Monthly Return' in df_period.columns:
        # 포트폴리오 가치 시계열 데이터 준비
        portfolio_values = df_period['Portfolio Value']

        # 전략 이름 생성 (팩터 이름과 전략 정보 결합)
        folder_name, file_name = os.path.split(csv_file)
        factor_name = folder_name
        strategy_info = file_name.replace('.csv', '')
        strategy_name = f"{factor_name} - {strategy_info}"

        # 그래프의 트레이스 생성 및 추가
        trace = go.Scatter(
            x=portfolio_values.index,
            y=portfolio_values.values,
            mode='lines',
            name=strategy_name
        )
        traces.append(trace)

        # Sharpe Ratio 계산
        sharpe_ratio = calculate_sharpe_ratio(df_period['Monthly Return'], rf_returns, training_start_date, training_end_date)

        # Sharpe Ratio를 리스트에 저장
        sharpe_ratio_list.append({
            'Strategy': strategy_name,
            'Sharpe Ratio': sharpe_ratio
        })
    else:
        print(f"'Portfolio Value' 또는 'Monthly Return' 컬럼이 없습니다: {csv_file}")

# 그래프 레이아웃 설정
layout = go.Layout(
    title='최적의 전략별 포트폴리오 가치 비교 (2008-10-31 ~ 2018-09-30)',
    xaxis=dict(title='날짜'),
    yaxis=dict(title='포트폴리오 가치'),
    hovermode='closest'
)

# 그래프 생성 및 출력
fig = go.Figure(data=traces, layout=layout)
fig.show()

# Sharpe Ratio를 데이터프레임으로 생성 및 출력
sharpe_ratio_df = pd.DataFrame(sharpe_ratio_list)
# print("\n각 전략의 연율화 Sharpe Ratio:")
# print(sharpe_ratio_df)

In [20]:
# 한 열에 표시되는 최대 문자 수 늘리기 (긴 문자열 표시)
pd.set_option('display.max_colwidth', None)
sharpe_ratio_df.to_csv('picked_strategy.csv', encoding='utf-8-sig')
sharpe_ratio_df

,Strategy,Sharpe Ratio
0,factor_betting_against_beta - rebalancing_1_threshold_0_3_cutoff_0_1_weighting_equal,0.654546
1,factor_high_pe_ratio - rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal,1.190827
2,factor_high_pe_ratio_large_industry - rebalancing_3_threshold_0_2_cutoff_0_1_weighting_equal,1.134090
3,factor_high_pe_ratio_medium_industry - rebalancing_1_threshold_0_2_cutoff_0_1_weighting_value,1.103080
4,factor_hml - rebalancing_3_threshold_0_2_cutoff_0_1_weighting_equal,1.007334
5,factor_momentum - rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal,0.951752
6,factor_momentum_large_industry - rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal,0.923269
7,factor_momentum_medium_industry - rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal,0.913603
8,factor_momentum_zscore - rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal,0.951752
9,factor_retained_earnings - rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal,1.122443


In [21]:
pd.reset_option('display.max_colwidth')

# CausalAI(였지만 지금은 그냥 선작업)

In [172]:
'''
2. 데이터 로드 및 전처리
2.1 선택된 백테스팅 CSV 파일 로드
'''
# 'best_results.txt' 파일을 읽어서 best_results 리스트를 생성
with open('best_results.txt', 'r', encoding='utf-8') as f:
    best_results = [line.strip() for line in f if line.strip()]

# 결과 출력
print("선택된 백테스팅 CSV 파일 목록:")
for file_path in best_results:
    print(file_path)

선택된 백테스팅 CSV 파일 목록:
factor_betting_against_beta\rebalancing_1_threshold_0_3_cutoff_0_1_weighting_equal.csv
factor_high_pe_ratio\rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal.csv
factor_high_pe_ratio_large_industry\rebalancing_3_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_high_pe_ratio_medium_industry\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_value.csv
factor_hml\rebalancing_3_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_momentum\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_momentum_large_industry\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_momentum_medium_industry\rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal.csv
factor_momentum_zscore\rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal.csv
factor_retained_earnings\rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal.csv


In [173]:
'''
2. 데이터 로드 및 전처리
2.2 그 외 데이터 로드
'''
# periods = [1,3,6,11]
periods = [12]

# 코스피/코스닥 지수 데이터 로드
market_indices = hs.load_market_indices()
market_indices.columns = market_indices.columns.droplevel('Symbol')

# 위험 무시 수익률 데이터 로드
rf_monthly = hs.load_risk_free_rate()

# 환율 데이터 로드
exchange_rate = hs.load_exchange_rate()

# 함수로 처리: 변화율 컬럼 추가
def add_pct_change(df, periods):
    """독립적으로 pct_change 값을 계산하고 데이터프레임에 추가"""
    new_df = df.copy()  # 원본 데이터를 변경하지 않기 위해 복사
    for period in periods:
        pct_change_data = df.pct_change(periods=period).shift(1)
        pct_change_data = pct_change_data.add_suffix(f'_pct_change_{period}m')
        new_df = new_df.join(pct_change_data, how='left')
    return new_df

# 변화율 계산 및 추가 (1개월, 3개월, 6개월, 12개월)
market_indices = add_pct_change(market_indices, periods)
rf_monthly = add_pct_change(rf_monthly, periods)
exchange_rate = add_pct_change(exchange_rate, periods)

# 날짜 정렬 및 결측치 처리
market_indices = market_indices.sort_index().fillna(method='ffill').dropna()
rf_monthly = rf_monthly.sort_index().fillna(method='ffill').dropna()
exchange_rate = exchange_rate.sort_index().fillna(method='ffill').dropna()

In [174]:
'''
2.3 전략 데이터 로드 및 합치기
'''
#딱 세 개만 사용 : 위의 전략 목록중 3개 임의로 선택
best_results = [best_results[1], best_results[5]
                # , best_results[9]
               ]

# 각 전략의 월별 수익률과 포트폴리오 가치를 저장할 데이터프레임 리스트 초기화
strategy_returns = []

# 각 CSV 파일을 읽어서 데이터프레임 생성
for csv_file in best_results:
    # CSV 파일 읽기
    df = pd.read_csv(csv_file, index_col=0, parse_dates=True)
    
    # 전략 이름 생성
    folder_name, file_name = os.path.split(csv_file)
    factor_name = folder_name
    strategy_info = file_name.replace('.csv', '')
    strategy_name = f"{factor_name}_{strategy_info}"
    
    # 'Monthly Return'과 'Portfolio Value' 컬럼 존재 여부 확인
    if 'Monthly Return' in df.columns and 'Portfolio Value' in df.columns:
        # 데이터프레임에 전략 이름을 접두사로 추가하여 컬럼 이름 변경
        df = df[['Monthly Return', 'Portfolio Value']].copy()
        df.columns = [f"{strategy_name}_Return", f"{strategy_name}_Value"]
        
        # 리스트에 추가
        strategy_returns.append(df)
    else:
        print(f"'Monthly Return' 또는 'Portfolio Value' 컬럼이 없습니다: {csv_file}")

# 모든 전략의 데이터프레임을 하나로 합치기
combined_df = pd.concat(strategy_returns, axis=1)

# 날짜로 정렬
combined_df = combined_df.sort_index()

# 수익률 데이터에 pct_change 계산 및 추가
return_columns = [col for col in combined_df.columns if '_Return' in col]
for period in periods:
    for col in return_columns:
        combined_df[f"{col}_pct_change_{period}m"] = combined_df[col].pct_change(periods=period)

# 누적 수익률 데이터에 대해 횡단면적 Z-Score 계산
value_columns = [col for col in combined_df.columns if '_Value' in col]

def calculate_zscore_over_periods(df, columns, periods):
    """기간별 Z-Score 계산"""
    for period in periods:
        # 숫자형 데이터만 선택
        numeric_columns = df[columns].select_dtypes(include=["number"])
        rolling_window = numeric_columns.rolling(window=period).mean()
        
        # Z-Score 계산
        zscore_df = rolling_window.apply(lambda x: zscore(x, nan_policy='omit') if len(x) > 1 else None, raw=False)
        zscore_df.columns = [f"{col}_zscore_{period}m" for col in numeric_columns.columns]

        # 직전 시점 값으로 이동
        zscore_df = zscore_df.shift(1)
        
        # 원본 데이터프레임에 추가
        df = df.join(zscore_df)
    return df

# Z-Score 계산 (3개월, 6개월, 12개월)
combined_df = calculate_zscore_over_periods(combined_df, value_columns, periods)

# 추가 데이터 합치기
combined_df = combined_df.join(market_indices, how='left')
combined_df = combined_df.join(rf_monthly, how='left')
combined_df = combined_df.join(exchange_rate, how='left')

# 결측치 처리 (앞 방향으로 채우기)
combined_df = combined_df.fillna(method='ffill').dropna()

# 결과 확인
print("\n합쳐진 데이터프레임의 일부:")
combined_df


합쳐진 데이터프레임의 일부:


,factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Return,factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Value,factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Return,factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Value,factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Return_pct_change_12m,factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Return_pct_change_12m,factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Value_zscore_12m,factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Value_zscore_12m,코스피,코스닥,코스피_pct_change_12m,코스닥_pct_change_12m,Risk_Free_Rate,Risk_Free_Rate_pct_change_12m,Exchange_Rate,Exchange_Rate_pct_change_12m
Date,,,,,,,,,,,,,,,,
2009-10-31,-0.032506,2.492724,-0.032034,1.557801,-inf,-inf,-1.380729,-1.640817,1580.69,486.46,0.155436,0.147855,3.510,-0.362162,1162.800049,-0.025813
2009-11-30,-0.016969,2.450425,-0.026380,1.516707,-1.897261,3.657964,-1.366047,-1.619876,1555.60,464.32,0.420130,0.579262,3.120,-0.283673,1160.599976,-0.098045
2009-12-31,0.121516,2.748192,0.112818,1.687818,-0.001345,2.706723,-1.351967,-1.600265,1682.77,513.57,0.445631,0.510082,3.480,-0.360656,1151.099976,-0.206373
2010-01-31,-0.025541,2.678001,-0.010893,1.669432,-1.268121,-1.301294,-1.336177,-1.575365,1602.43,496.57,0.496501,0.546665,3.220,0.067485,1147.699951,-0.073338
2010-02-28,0.028257,2.753672,0.049845,1.752645,2.638953,3.372724,-1.322149,-1.552547,1594.58,507.03,0.378897,0.360839,3.070,0.272727,1133.599976,-0.154113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-31,-0.015329,30.855714,-0.010910,8.425144,-0.479723,-3.547213,1.863306,1.361817,2770.69,803.15,0.091074,-0.032019,3.152,-0.065956,1383.500000,0.043240
2024-08-31,-0.032264,29.860183,-0.024565,8.218183,-3.604592,-7.631673,1.891093,1.382030,2674.31,767.66,0.052462,-0.141906,3.060,-0.105308,1336.569946,0.087324
2024-09-30,0.004362,29.990448,-0.005768,8.170785,-1.140813,-0.801113,1.905673,1.393377,2593.27,763.88,0.046177,-0.173137,2.826,-0.125714,1309.300049,0.010234


In [176]:
'''
3. 레이블 생성
'''
# 각 시점마다 가장 높은 수익률을 보인 전략의 이름을 레이블로 생성
return_columns = [col for col in combined_df.columns if 'Return' in col and 'pct_change' not in col and 'STD' not in col]
combined_df['Best_Strategy'] = combined_df[return_columns].idxmax(axis=1)

# 레이블 인코딩
label_encoder = LabelEncoder()
combined_df['Best_Strategy_Label'] = label_encoder.fit_transform(combined_df['Best_Strategy'])

# 결과 확인
print("\n레이블이 추가된 데이터프레임의 일부:")
combined_df[['Best_Strategy', 'Best_Strategy_Label']]


레이블이 추가된 데이터프레임의 일부:


,Best_Strategy,Best_Strategy_Label
Date,,
2009-10-31,factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Return,1
2009-11-30,factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Return,0
2009-12-31,factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Return,0
2010-01-31,factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Return,1
2010-02-28,factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Return,1
...,...,...
2024-07-31,factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Return,1
2024-08-31,factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Return,1
2024-09-30,factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Return,0


In [178]:
train_start_date = pd.to_datetime('2007-11-30')
train_end_date = pd.to_datetime('2018-10-31')
val_start_date = pd.to_datetime('2018-11-30')
val_end_date = pd.to_datetime('2021-10-31')
test_start_date = pd.to_datetime('2021-11-30')
test_end_date = pd.to_datetime('2024-10-31')

# Train, validation, test 데이터 나누기
train_data = combined_df[(combined_df.index >= train_start_date) & (combined_df.index <= train_end_date)]
val_data = combined_df[(combined_df.index >= val_start_date) & (combined_df.index <= val_end_date)]
test_data = combined_df[(combined_df.index >= test_start_date) & (combined_df.index <= test_end_date)]

# 각 데이터셋의 샘플 개수 출력
print(f"Train 데이터 개수: {len(train_data)}")
print(train_data['Best_Strategy_Label'].value_counts())
print(f"Validation 데이터 개수: {len(val_data)}")
print(val_data['Best_Strategy_Label'].value_counts())
print(f"Test 데이터 개수: {len(test_data)}")
print(test_data['Best_Strategy_Label'].value_counts())

Train 데이터 개수: 109
Best_Strategy_Label
0    59
1    50
Name: count, dtype: int64
Validation 데이터 개수: 36
Best_Strategy_Label
0    21
1    15
Name: count, dtype: int64
Test 데이터 개수: 36
Best_Strategy_Label
0    21
1    15
Name: count, dtype: int64


# 딥-러닝

In [179]:
###################################
# 0. 가정된 전역 변수들 (사용 예)
###################################
# 날짜 범위
train_start_date = pd.to_datetime('2007-11-30')
train_end_date = pd.to_datetime('2018-10-31')
val_start_date = pd.to_datetime('2018-11-30')
val_end_date = pd.to_datetime('2021-10-31')
test_start_date = pd.to_datetime('2021-11-30')
test_end_date = pd.to_datetime('2024-10-31')

label_col = 'Best_Strategy_Label'
strategy_return_cols = [
    'factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Return',
    'factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Return'
    # 'factor_retained_earnings_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Return'
]

num_classes = len(strategy_return_cols)

# feature_cols에서 다음을 제외:
# - 타겟 라벨(Best_Strategy_Label)
# - 직접적으로 전략 수익률 컬럼(strategy_return_cols)
# - Best_Strategy (라벨과 유사, 미래예측에 활용 불가한 정보)
#
# pct_change나 Value_zscore 컬럼도 사용해볼 수 있으므로 제외 조건 제거
feature_cols = [
    col for col in combined_df.columns 
    if col not in [label_col, 'Best_Strategy'] + strategy_return_cols
]

# Hyperparameters
lookback = 12
batch_size = 64
epochs = 80
learning_rate = 0.001
use_gru = True  # True면 GRU, False면 LSTM
use_diff = False  # 비정상성 데이터 차분 적용여부
diff_cols = []    # 차분 적용할 컬럼 리스트
use_pca = False    # PCA 적용 여부
n_pca_components = 5
use_oversampling = False  # 오버샘플링 여부 (ADASYN)
unit = 16

In [180]:
feature_cols

['factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Value',
 'factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Value',
 'factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Return_pct_change_12m',
 'factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Return_pct_change_12m',
 'factor_high_pe_ratio_rebalancing_1_threshold_0_1_cutoff_0_05_weighting_equal_Value_zscore_12m',
 'factor_momentum_rebalancing_1_threshold_0_2_cutoff_0_1_weighting_equal_Value_zscore_12m',
 '코스피',
 '코스닥',
 '코스피_pct_change_12m',
 '코스닥_pct_change_12m',
 'Risk_Free_Rate',
 'Risk_Free_Rate_pct_change_12m',
 'Exchange_Rate',
 'Exchange_Rate_pct_change_12m']

In [181]:
###################################
# 함수 정의 (반복적으로 사용하는 것만)
###################################
def custom_loss_fn(y_true, y_pred, returns):
    actual_return = tf.reduce_sum(y_true * returns, axis=1)
    predicted_return = tf.reduce_sum(y_pred * returns, axis=1)
    return tf.reduce_mean(tf.square(actual_return - predicted_return))

@tf.function
def train_step(model, optimizer, x, y, r):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)
        loss_value = custom_loss_fn(y, y_pred, r)
    grads = tape.gradient(loss_value, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss_value

@tf.function
def val_step(model, x, y, r):
    y_pred = model(x, training=False)
    loss_value = custom_loss_fn(y, y_pred, r)
    return loss_value, y_pred

def scale_returns(train_ret, val_ret, test_ret):
    scaler = StandardScaler()
    train_ret_scaled = scaler.fit_transform(train_ret)
    val_ret_scaled = scaler.transform(val_ret)
    test_ret_scaled = scaler.transform(test_ret)
    return train_ret_scaled, val_ret_scaled, test_ret_scaled

In [182]:
###################################
# 메인 코드 시작
###################################

# 1. 차분 적용 (if use_diff)
if use_diff and len(diff_cols) > 0:
    df = combined_df.copy()
    for c in diff_cols:
        if c in df.columns:
            df[c+'_diff'] = df[c].diff()
    df.dropna(inplace=True)
    # 차분 적용 시, feature_cols 갱신
    for c in diff_cols:
        if c in df.columns and c+'_diff' in df.columns:
            feature_cols.append(c+'_diff')
    # 원본 컬럼을 유지하거나 제거할지 결정
    # 여기서는 원본 유지 + diff컬럼 추가한 상태로 진행
    combined_df = df
else:
    # 차분 적용 안함
    pass

In [183]:
# 2. Train/Val/Test split
train_df = combined_df.loc[train_start_date:train_end_date]
val_df = combined_df.loc[val_start_date:val_end_date]
test_df = combined_df.loc[test_start_date:test_end_date]

In [184]:
# 차분, PCA 등 전처리 완료 후 스케일링 전 단계
train_df[feature_cols] = train_df[feature_cols].replace([np.inf, -np.inf], np.nan)
val_df[feature_cols] = val_df[feature_cols].replace([np.inf, -np.inf], np.nan)
test_df[feature_cols] = test_df[feature_cols].replace([np.inf, -np.inf], np.nan)

# 결측치 드롭 (필요 시 전략적으로 처리)
train_df = train_df.dropna(subset=feature_cols)
val_df = val_df.dropna(subset=feature_cols)
test_df = test_df.dropna(subset=feature_cols)

# 3. PCA 적용 (if use_pca)
# PCA는 (samples, features) 형태로 변환 후 다시 dataframe에 붙여야 함
if use_pca:
    pca_scaler = StandardScaler()
    train_scaled = pca_scaler.fit_transform(train_df[feature_cols])
    pca = PCA(n_components=n_pca_components)
    train_pca = pca.fit_transform(train_scaled)

    # validation, test PCA 적용
    val_scaled = pca_scaler.transform(val_df[feature_cols])
    val_pca = pca.transform(val_scaled)
    test_scaled = pca_scaler.transform(test_df[feature_cols])
    test_pca = pca.transform(test_scaled)

    # PCA 컴포넌트를 DF에 붙이기
    pca_cols = [f'PCA_{i+1}' for i in range(n_pca_components)]
    for i, col in enumerate(pca_cols):
        train_df[col] = train_pca[:, i]
        val_df[col] = val_pca[:, i]
        test_df[col] = test_pca[:, i]

    # feature_cols 업데이트
    feature_cols = feature_cols + pca_cols
else:
    # PCA 미적용 시
    pass

In [185]:
# 4. Scaling feature_cols
scaler = StandardScaler()
train_X = scaler.fit_transform(train_df[feature_cols])
val_X = scaler.transform(val_df[feature_cols])
test_X = scaler.transform(test_df[feature_cols])

# 라벨, 리턴 추출
train_y = train_df[label_col].values
val_y = val_df[label_col].values
test_y = test_df[label_col].values

train_r = train_df[strategy_return_cols].values
val_r = val_df[strategy_return_cols].values
test_r = test_df[strategy_return_cols].values

# 수익률 스케일링
train_r, val_r, test_r = scale_returns(train_r, val_r, test_r)

In [186]:
# 5. create_sequences
def create_sequences(X, y, returns, lookback=12):
    X_seq, y_seq, ret_seq = [], [], []
    for i in range(len(X)-lookback):
        X_seq.append(X[i:i+lookback])
        y_seq.append(y[i+lookback])
        ret_seq.append(returns[i+lookback])
    return np.array(X_seq), np.array(y_seq), np.array(ret_seq)

train_X_seq, train_y_seq, train_ret_seq = create_sequences(train_X, train_y, train_r, lookback)
val_X_seq, val_y_seq, val_ret_seq = create_sequences(val_X, val_y, val_r, lookback)
test_X_seq, test_y_seq, test_ret_seq = create_sequences(test_X, test_y, test_r, lookback)

# One-hot label
train_y_cat = tf.keras.utils.to_categorical(train_y_seq, num_classes=num_classes)
val_y_cat = tf.keras.utils.to_categorical(val_y_seq, num_classes=num_classes)
test_y_cat = tf.keras.utils.to_categorical(test_y_seq, num_classes=num_classes)

train_X_seq = train_X_seq.astype(np.float32)
train_y_cat = train_y_cat.astype(np.float32)
train_ret_seq = train_ret_seq.astype(np.float32)

val_X_seq = val_X_seq.astype(np.float32)
val_y_cat = val_y_cat.astype(np.float32)
val_ret_seq = val_ret_seq.astype(np.float32)

test_X_seq = test_X_seq.astype(np.float32)
test_y_cat = test_y_cat.astype(np.float32)
test_ret_seq = test_ret_seq.astype(np.float32)

In [187]:
# 6. 오버샘플링 (if use_oversampling)
if use_oversampling:
    # ADASYN은 2D 데이터 필요 -> (samples, timesteps*features)로 변경
    orig_shape = train_X_seq.shape
    samples, timesteps, features = orig_shape
    X_2d = train_X_seq.reshape(samples, timesteps*features)
    y_int = np.argmax(train_y_cat, axis=1)  # One-hot -> int label

    adasyn = ADASYN()
    X_res, y_res = adasyn.fit_resample(X_2d, y_int)
    # 다시 3D로 변환
    new_samples = X_res.shape[0]
    X_res_3d = X_res.reshape(new_samples, timesteps, features)
    # returns도 oversampling?
    # oversampling 라벨 개수 늘어났으니 return도 필요. 
    # 하지만 return은 라벨과 1:1이므로 y와 같이 oversample 해야함.
    # y_res 인덱스를 활용하기 위해 인덱스 매핑 필요
    # 이건 실제 구현 시 주의 필요. 여기서는 라벨과 동일 인덱스로 mapping된다고 가정불가.
    # 간단히는 오버샘플링 전후 라벨 순서 유지 불가 -> ADASYN 후 y_res 인덱스로 train_ret_seq도 재배열 불가
    # 실제론 y_res의 각 샘플이 기존 어떤 샘플에서 생성됐는지 추적 필요.
    # 여기서는 단순 예시로 오버샘플링 시 returns는 적용하지 않고 pass(실무에서는 재설계 필요)
    # 오버샘플링은 시계열에 부자연스러우므로 여기선 skip을 권장하지만, 요구사항이므로 예제만.
    # 만약 return도 oversample하려면 같은 random_state로 ADASYN fitter를 사용해 
    # 라벨별 인덱스 추적 후 returns 재배치를 해야함.
    # 여기선 간략히 returns 재매핑 안함 -> 실제론 불일치 발생, 이건 구현상 한계
    
    # Warning: 아래 코드는 oversampling과 returns 일치성 문제 있음(단순히 same indexing이라 가정)
    # 실제로는 returns도 fit_resample 해야 하나, ADASYN은 y 기반이므로 returns 다룰때 주의 필요.
    # 여기서는 가상의 로직 (returns도 y와 동일 indexing이라 가정)
    train_ret_seq = train_ret_seq[y_int.argsort()][y_res.argsort()] # 이건 가짜로 만든 예
    # 이 부분 실제로는 별도 로직 필요. 여기선 단순히 형식 맞추기 어려움
    # 실제로는 oversampling이 시계열에 부적절할 수 있음.

    train_X_seq = X_res_3d
    train_y_cat = tf.keras.utils.to_categorical(y_res, num_classes=num_classes)
else:
    pass

In [188]:
# 7. 모델 정의
model = Sequential()
if use_gru:
    model.add(GRU(units=unit, activation='elu', input_shape=(lookback, len(feature_cols))))
else:
    model.add(LSTM(units=unit, activation='elu', input_shape=(lookback, len(feature_cols))))
model.add(Dense(num_classes, activation='softmax'))

optimizer = Adam(learning_rate=learning_rate)

In [189]:
# 8. Dataset 준비
train_dataset = tf.data.Dataset.from_tensor_slices((train_X_seq, train_y_cat, train_ret_seq)).shuffle(1024).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((val_X_seq, val_y_cat, val_ret_seq)).batch(batch_size)

# 여기서 더미 인퍼런스 한 번 실행 (train_dataset에서 하나 추출)
# Creating variables on a non-first call to a function decorated with tf.function 에러 방지용
x_dummy, y_dummy, r_dummy = next(iter(train_dataset))
model(x_dummy)  # 이렇게 한 번 호출해서 변수 생성 완료

best_val_loss = np.inf

for epoch in range(epochs):
    train_losses = []
    for x_batch, y_batch, r_batch in train_dataset:
        loss_value = train_step(model, optimizer, x_batch, y_batch, r_batch)
        train_losses.append(loss_value.numpy())

    val_losses = []
    for x_batch, y_batch, r_batch in val_dataset:
        val_loss_value, _ = val_step(model, x_batch, y_batch, r_batch)
        val_losses.append(val_loss_value.numpy())

    val_loss_mean = np.mean(val_losses)
    train_loss_mean = np.mean(train_losses)
    # if epoch%10==0:
    print(f"Epoch {epoch+1}, Train Loss: {train_loss_mean:.4f}, Val Loss: {val_loss_mean:.4f}")

    if val_loss_mean < best_val_loss:
        best_val_loss = val_loss_mean
        model.save('best_model.keras')

Epoch 1, Train Loss: 0.0628, Val Loss: 0.4881
Epoch 2, Train Loss: 0.0676, Val Loss: 0.4914
Epoch 3, Train Loss: 0.0660, Val Loss: 0.4897
Epoch 4, Train Loss: 0.0593, Val Loss: 0.4895
Epoch 5, Train Loss: 0.0634, Val Loss: 0.4903
Epoch 6, Train Loss: 0.0564, Val Loss: 0.4913
Epoch 7, Train Loss: 0.0601, Val Loss: 0.4931
Epoch 8, Train Loss: 0.0638, Val Loss: 0.4944
Epoch 9, Train Loss: 0.0609, Val Loss: 0.4960
Epoch 10, Train Loss: 0.0617, Val Loss: 0.4990
Epoch 11, Train Loss: 0.0585, Val Loss: 0.4992
Epoch 12, Train Loss: 0.0632, Val Loss: 0.5005
Epoch 13, Train Loss: 0.0598, Val Loss: 0.5009
Epoch 14, Train Loss: 0.0575, Val Loss: 0.5015
Epoch 15, Train Loss: 0.0615, Val Loss: 0.5033
Epoch 16, Train Loss: 0.0569, Val Loss: 0.5028
Epoch 17, Train Loss: 0.0613, Val Loss: 0.5038
Epoch 18, Train Loss: 0.0596, Val Loss: 0.5057
Epoch 19, Train Loss: 0.0563, Val Loss: 0.5080
Epoch 20, Train Loss: 0.0583, Val Loss: 0.5117
Epoch 21, Train Loss: 0.0592, Val Loss: 0.5157
Epoch 22, Train Loss: 